# A Notebook to begin Filtering RFI

This is the second notebook that I made while working on ARTISTIC. It is still a little bit less focused than some of the other notebooks that come after this one because I did not have as clear of an idea about how I would go about narrowing down 5.7 million hits to a handful of hits to be able to comb through individually. This notebook starts out by going through and taking out all of the hits that were present in all six beams at the same frequency in the same field of view. This is one of the things that I practiced doing in the Practice Hits Notebook. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from tabulate import tabulate
from tqdm import tqdm

In [3]:
df = pd.read_json('../../Preliminary_Data_Files/hits_collation.json')
print(df.shape)

(5697746, 24)


In [4]:
######### Looking at the number of hits in each field of view #########
file_path = df["file_path"]
unique_file_path = np.unique(file_path) #the array of unique fields of view
print(len(unique_file_path))
print(len(np.unique(df["tstart"])))

22826
1721


In [ ]:
######### Start by making a blank data frame that we can append information to #########
df1 = pd.DataFrame({"file_path":[],
                    "hit_file_enumeration":[],
                    "signal_frequency":[],
                    "signal_index":[],
                    "signal_driftSteps":[],
                    "signal_driftRate":[],
                    "signal_snr":[],
                    "signal_coarseChannel":[],
                    "signal_numTimesteps":[],
                    "signal_power":[],
                    "signal_incoherentPower":[],
                    "sourceName":[],
                    "fch1":[],
                    "foff":[],
                    "tstart":[],
                    "tsamp":[],
                    "ra":[],
                    "dec":[],
                    "telescopeId":[],
                    "numTimesteps":[],
                    "numChannels":[],
                    "coarseChannel":[],
                    "startChannel":[],
                    "beam":[]})

######## This for loop should go through and add all of the tables that have the beam anomalies to a new table #########
for i in tqdm(range(len(unique_file_path))):
#for i in range(1):
    fov_subset = df.loc[df['file_path'] == unique_file_path[i]] #select each subset
    fov_subset = fov_subset.sort_values(by = ["signal_frequency"])
    
    freq = fov_subset["signal_frequency"] #define just the column for frequencies
    power = fov_subset["signal_power"] #define just the column for power
    unique_freq = np.unique(freq) #find the array of unique frequencies

    #Here we want to find how many hits each frequency had between the 5 coherent beams and the one incoherent beam
    hits_per_freq = []
    for k in range(len(unique_freq)):
        hits_per_freq.append(np.count_nonzero(freq == unique_freq[k]))
    #print(hits_per_freq)

    #Each frequency should have a maximum of 6 hits if it is widespread RFI
    indices = np.where(np.array(hits_per_freq) != 6)
    values_less_than_6 = np.array(unique_freq)[indices]
    
    new_indices = np.concatenate(np.where(np.isin(np.array(freq), values_less_than_6) == True))
    
    appending_rows = fov_subset.iloc[new_indices]
    
    df1 = df1.append(appending_rows, ignore_index = True)
display(df1)

In [ ]:
with open('subset_3.pkl', 'wb') as f:  # open a text file
    pickle.dump(df1, f) # serialize the list
f.close()

In [ ]:
start_time = []
#for i in range(len(unique_file_path)):
for i in range(100):
    fov_subset = df.loc[df['file_path'] == unique_file_path[i]] #select each subset
    
    freq = fov_subset["signal_frequency"] #define just the column for frequencies
    power = fov_subset["signal_power"] #define just the column for power
    unique_freq = np.unique(freq) #find the array of unique frequencies

    #Here we want to find how many hits each frequency had between the 5 coherent beams and the one incoherent beam
    hits_per_freq = []
    for k in range(len(unique_freq)):
        hits_per_freq.append(np.count_nonzero(freq == unique_freq[k]))

    #Each frequency should have a maximum of 6 hits if it is widespread RFI
    indices = np.where(np.array(hits_per_freq) < 6)
    #print(indices)
    values_less_than_6 = np.array(unique_freq)[indices]
    #print(values_less_than_6)
    #print(np.array(hits_per_freq)[indices])
    
    new_indices = np.where(np.isin(np.array(fov_subset.sort_values(by = ["signal_frequency", "beam"])["signal_frequency"]), 
                      values_less_than_6) == True)
    
    print(new_indices)
    
    time = np.array(fov_subset.sort_values(by = ["signal_frequency", "beam"])["tstart"])[new_indices]
    start_time.append(time)
    
    print(i, ' out of ', len(unique_file_path))
start_time  = np.concatenate(start_time)
print(len(np.unique(start_time)))

Now open up that .pickle file and make sure that it is everything that we were trying to get

Printing out some things just to see the shape of the data we are working with. Does not need to be run each time because nothing gets defined.

Now just looking at a sample plot to see what the frequencies of the hits are. Also does not need to be run each time.

In [ ]:
sns.set_theme()
g = sns.relplot(data=df1, x=df1.iloc[:]["ra"], y=df1.iloc[:]["dec"], hue=df1.iloc[:]["signal_frequency"])

Define a few variables. Should be run each time because a lot of things reference this one.

In [ ]:
source_name = np.unique(subset_2["Source Name"]) #The names of the 7732 unique sources (plus the incoherent beam)
ra = subset_2["Right Ascension"]
dec = subset_2["Declination"]

print(len(source_name))

Open the pickle file that has the incorrect information for the number of fields per unique source. Could honestly be removed because that file is not correct.

In [ ]:
with open('unique_fields_1.pkl', 'wb') as f:  # open a text file
    pickle.dump(number_of_fovs, f) # serialize the list
f.close()

Arrange the information in the subset pickle file based on a few characteristics. We will filter based on this information. It could be a good idea to toss this into a pickle file because it tends to take a while to run.

In [ ]:
info = pd.DataFrame({'Source': np.array(subset_1.sort_values(by = ["Source Name", "File Path"])["Source Name"]), 
        'File Path': np.array(subset_1.sort_values(by = ["Source Name", "File Path"])["File Path"]), 
        'Right Ascension': np.array(subset_1.sort_values(by = ["Source Name", "File Path"])["Right Ascension"]), 
       'Declination': np.array(subset_1.sort_values(by = ["Source Name", "File Path"])["Declination"])})

display(info)

This sets up the indexing and figuring out how many unique fields of view there are per source. It takes a long time to run and because the information is already calculated and being stored in a pickle file, should not be run again.

In [ ]:
source_name = np.unique(subset_1["Source Name"])
source = subset_1["Source Name"]

hits_per_source = []
for i in range(len(source_name)):
    hits_per_source.append(np.count_nonzero(source == source_name[i]))

In [ ]:
print(len(source_name), len(source))

print(hits_per_source[0])

We might need the indices again and this one doesn't take long to run, so it should be run every time.

In [ ]:
indices = [0]
for i in range(len(hits_per_source)-1):
    indices.append(indices[i] + hits_per_source[i])
    
print(len(indices))

Do not run this again. All of the information has already been stored in a pickle file, so you just need to open the file.

In [ ]:
hits_info = pd.DataFrame({'Source': np.array(subset_1.sort_values(by = ["Source Name", "File Path"])["Source Name"])[indices], 
        'Number of Hits': np.array(hits_per_source), 
        'Right Ascension': np.array(subset_1.sort_values(by = ["Source Name", "File Path"])["Right Ascension"])[indices], 
       'Declination': np.array(subset_1.sort_values(by = ["Source Name", "File Path"])["Declination"])[indices]})

display(hits_info)

#with open('hits_per_source.pkl', 'wb') as f:  # open a text file
    #pickle.dump(hits_info, f) # serialize the list
#f.close()

Be sure to open this each time. The naming conventions are the the same for below so there should be no issues.

In [ ]:
with open('hits_per_source.pkl', 'rb') as f:
    hits_info = pickle.load(f)

In [ ]:
sns.set_theme()
sns.relplot(data=hits_info[0:-1], x=hits_info["Right Ascension"][0:-1], y=hits_info["Declination"][0:-1], 
            hue=hits_info["Number of Hits"][0:-1], size = hits_info["Number of Hits"][0:-1]

In [ ]:
from tqdm.notebook import tqdm
from time import sleep

def countFreq(arr, n):
    
    # Mark all array elements as not visited
    visited = [False for i in range(n)]
    fp_list = []
    # Traverse through array elements and count frequencies
    for i in range(n):
         
        # Skip this element if already processed
        if (visited[i] == True):
            continue
 
        # Count frequency
        count = 1
        for j in range(i + 1, n, 1):
            if (arr[i] == arr[j]):
                visited[j] = True
                count += 1
         
        #print(count)
        fp_list.append(count)
    return len(fp_list)
        
#Driver Code
arr = np.array(info["File Path"][693038:-1])
n = len(arr)
countFreq(arr, n)
#countFreq(np.array(info["File Path"]), len(np.array(info["File Path"])))

In [ ]:
fovs_per_source = []
for i in range(len(indices)-1):
    arr = np.array(info["File Path"][indices[i]:indices[i+1]])
    n = len(arr)
    fovs_per_source.append(countFreq(arr, n))
#print(len(fovs_per_source))
fovs_per_source.append(18194)

for i in tqdm(range(100)):
    sleep(3)
#print(indices[-1])

In [ ]:
fovs_info = pd.DataFrame({'Source': np.array(subset_1.sort_values(by = ["Source Name", "File Path"])["Source Name"])[indices], 
        'Number of FOVs': np.array(fovs_per_source), 
        'Right Ascension': np.array(subset_1.sort_values(by = ["Source Name", "File Path"])["Right Ascension"])[indices], 
       'Declination': np.array(subset_1.sort_values(by = ["Source Name", "File Path"])["Declination"])[indices]})

display(fovs_info)

with open('fovs_per_source.pkl', 'wb') as f:  # open a text file
    pickle.dump(fovs_info, f) # serialize the list
f.close()

In [ ]:
display(fovs_info[0:50])

In [ ]:
with open('fovs_per_source.pkl', 'rb') as f:
    fovs_info = pickle.load(f)

In [ ]:
sns.set_theme()
sns.relplot(data=fovs_info[0:-1], x=fovs_info["Right Ascension"][0:-1], y=fovs_info["Declination"][0:-1], 
            hue=fovs_info["Number of FOVs"][0:-1], size = fovs_info["Number of FOVs"][0:-1], palette = "ch:s=.25,rot=-.25")

#plt.savefig('FOVs_per_Source.png', bbox_inches='tight', transparent = True)

In [ ]:
print(np.max(fovs_info["Number of FOVs"][0:-1]))
print(np.min(fovs_info["Number of FOVs"][0:-1]))

Ok now we need to start looking through each of the fields of view to see where there might be similar frequencies and characteristics. So let's start by trying to just pull all of the data for one source with a lot of fields of view. By looking at the table above, I'm going to start by just choosing a source.

In [ ]:
source_1 = subset_1.loc[subset_1["Source Name"] == "10132416262554752"]

display(source_1[0:20])

print(np.array(source_1["Signal to Noise"]))

print(len(subset_1["Signal Index"]))

In [ ]:
print(len(np.unique(source_1["Frequency"])))
print(len(source_1["Frequency"]))
#print(np.array(source_1["Frequency"]))

def countFreq(arr, n):
    
    # Mark all array elements as not visited
    visited = [False for i in range(n)]
    # Traverse through array elements and count frequencies
    for i in range(n):
         
        # Skip this element if already processed
        if (visited[i] == True):
            continue
 
        # Count frequency
        count = 1
        for j in range(i + 1, n, 1):
            if (arr[i] == arr[j]):
                visited[j] = True
                count += 1
         
        print(arr[i], count)
        #fp_list.append(count)
        
#Driver Code
arr = np.array(source_1["Frequency"])
n = len(arr)
countFreq(arr, n)

In [ ]:
sns.set_theme()
sns.relplot(data=source_1, x=source_1["Frequency"], y=source_1["Drift Rate"], 
            hue=source_1["Signal to Noise"], size = source_1["Signal to Noise"])